# Prepare data, install libraries

In [ ]:
!mkdir /content/DeepFashion2/

from shutil import copyfile
src = '/content/drive/MyDrive/Colab Notebooks/train.zip'
dst = '/content/DeepFashion2/train.zip'
copyfile(src, dst)

mkdir: cannot create directory ‘/content/DeepFashion2/’: File exists


'/content/DeepFashion2/train.zip'

In [ ]:
src = '/content/drive/MyDrive/Colab Notebooks/validation.zip'
dst = '/content/DeepFashion2/validation.zip'
copyfile(src, dst)

'/content/DeepFashion2/validation.zip'

In [ ]:
!unzip -P 2019Deepfashion2** /content/DeepFashion2/train.zip -d /content/DeepFashion2/
!unzip -P 2019Deepfashion2** /content/DeepFashion2/validation.zip -d /content/DeepFashion2/

串流輸出內容已截斷至最後 5000 行。
  inflating: /content/DeepFashion2/validation/image/007349.jpg  
  inflating: /content/DeepFashion2/validation/image/007310.jpg  
  inflating: /content/DeepFashion2/validation/image/007286.jpg  
  inflating: /content/DeepFashion2/validation/image/007274.jpg  
  inflating: /content/DeepFashion2/validation/image/007218.jpg  
  inflating: /content/DeepFashion2/validation/image/007216.jpg  
  inflating: /content/DeepFashion2/validation/image/007204.jpg  
  inflating: /content/DeepFashion2/validation/image/007133.jpg  
  inflating: /content/DeepFashion2/validation/image/007081.jpg  
  inflating: /content/DeepFashion2/validation/image/007045.jpg  
  inflating: /content/DeepFashion2/validation/image/007044.jpg  
  inflating: /content/DeepFashion2/validation/image/007017.jpg  
  inflating: /content/DeepFashion2/validation/image/006960.jpg  
  inflating: /content/DeepFashion2/validation/image/006868.jpg  
  inflating: /content/DeepFashion2/validation/image/006831.jpg  
  in

In [ ]:
!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/index.html

Looking in links: https://download.pytorch.org/whl/cu101/torch_stable.html
     |████████████████████████████████| 703.8MB 24kB/s 
     |████████████████████████████████| 6.6MB 36.4MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101
     |████████████████████████████████| 276kB 13.4MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.1-cp36-cp36m-linux_x86_64.whl size=44076 sha256=a1e4dad60d4c7f2e41f5d595ab42657f6a97ff3660b1947f9d8e0e9f8a10e0bd
  Stored in directory: /root/.cache/pip/wheels/ad/56/bc/1522f864feb2a358ea6f1a92b4798d69ac783a28e80567a18b
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pi

# **We create a json file here for validation**

In [ ]:
from PIL import Image
import numpy as np
import json

dataset = {
    "info": {},
    "licenses": [],
    "images": [],
    "annotations": [],
    "categories": []
}

lst_name = ['short_sleeved_shirt', 'long_sleeved_shirt', 'short_sleeved_outwear', 'long_sleeved_outwear',
            'vest', 'sling', 'shorts', 'trousers', 'skirt', 'short_sleeved_dress',
            'long_sleeved_dress', 'vest_dress', 'sling_dress']

for idx, e  in enumerate(lst_name):
    dataset['categories'].append({
        'id': idx + 1,
        'name': e,
        'supercategory': "clothes",
        'keypoints': ['%i' % (i) for i in range(1, 295)],
        'skeleton': []
    })

num_images = 5000 #191961 #32153
sub_index = 0  # the index of ground truth instance
for num in range(1, num_images + 1):
    json_name = '/content/DeepFashion2/validation/annos/' + str(num).zfill(6) + '.json'
    image_name = '/content/DeepFashion2/validation/image/' + str(num).zfill(6) + '.jpg'

    if (num >= 0):
        imag = Image.open(image_name)
        width, height = imag.size
        with open(json_name, 'r') as f:
            temp = json.loads(f.read())
            pair_id = temp['pair_id']

            dataset['images'].append({
                'coco_url': '',
                'date_captured': '',
                'file_name': str(num).zfill(6) + '.jpg',
                'flickr_url': '',
                'id': num,
                'license': 0,
                'width': width,
                'height': height
            })
            for i in temp:
                if i == 'source' or i == 'pair_id':
                    continue
                else:
                    points = np.zeros(294 * 3)
                    sub_index = sub_index + 1
                    box = temp[i]['bounding_box']
                    w = box[2] - box[0]
                    h = box[3] - box[1]
                    x_1 = box[0]
                    y_1 = box[1]
                    bbox = [x_1, y_1, w, h]
                    cat = temp[i]['category_id']
                    style = temp[i]['style']
                    seg = temp[i]['segmentation']
                    landmarks = temp[i]['landmarks']

                    points_x = landmarks[0::3]
                    points_y = landmarks[1::3]
                    points_v = landmarks[2::3]
                    points_x = np.array(points_x)
                    points_y = np.array(points_y)
                    points_v = np.array(points_v)
                    case = [0, 25, 58, 89, 128, 143, 158, 168, 182, 190, 219, 256, 275, 294]
                    idx_i, idx_j = case[cat - 1], case[cat]

                    for n in range(idx_i, idx_j):
                        points[3 * n] = points_x[n - idx_i]
                        points[3 * n + 1] = points_y[n - idx_i]
                        points[3 * n + 2] = points_v[n - idx_i]

                    num_points = len(np.where(points_v > 0)[0])

                    dataset['annotations'].append({
                        'area': w * h,
                        'bbox': bbox,
                        'category_id': cat,
                        'id': sub_index,
                        'pair_id': pair_id,
                        'image_id': num,
                        'iscrowd': 0,
                        'style': style,
                        'num_keypoints': num_points,
                        'keypoints': points.tolist(),
                        'segmentation': seg,
                    })

json_name = '/content/deepfashion2_validation.json'
with open(json_name, 'w') as f:
    json.dump(dataset, f)

# **Import libraries**

In [ ]:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import os
import numpy as np
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

** fvcore version of PathManager will be deprecated soon. **
** Please migrate to the version in iopath repo. **
https://github.com/facebookresearch/iopath 



# **Register coco instances for Training  and Validation**

In [ ]:
from detectron2.data.datasets import register_coco_instances
register_coco_instances("deepfashion_train_1_100000", {}, "/content/deepfashion2_train_1_100000.json", "/content/DeepFashion2/train/image")
register_coco_instances("deepfashion_val", {}, "/content/deepfashion2_validation.json", "/content/DeepFashion2/validation/image")

# **Setup configuration for first time training**

In [ ]:
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("deepfashion_train_1_30000",)
cfg.DATASETS.TEST = ()
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.MODEL.WEIGHTS = '/content/DeepFashion2/model_final202012111330.pth'

cfg.SOLVER.IMS_PER_BATCH = 8
cfg.SOLVER.BASE_LR = 0.0025 #0.001
cfg.SOLVER.WARMUP_ITERS = 500
cfg.SOLVER.MAX_ITER = 4000
cfg.SOLVER.STEPS = (500,2000,3500)
cfg.SOLVER.GAMMA = 0.1
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 13

cfg.TEST.EVAL_PERIOD = 2000

# First Training & evaluate

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[12/16 02:48:38 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

In [ ]:
#save weight
from shutil import copyfile

src = '/content/output/model_final.pth'
dst = '/content/drive/MyDrive/Colab Notebooks/model_final_202012161200.pth'
copyfile(src, dst)

'/content/drive/MyDrive/Colab Notebooks/model_final_202012161200.pth'

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

evaluator = COCOEvaluator("deepfashion_val", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "deepfashion_val")
result=inference_on_dataset(trainer.model, val_loader, evaluator)

[12/16 04:01:02 d2.data.datasets.coco]: Loading /content/deepfashion2_validation.json takes 1.56 seconds.
[12/16 04:01:02 d2.data.datasets.coco]: Loaded 5000 images in COCO format from /content/deepfashion2_validation.json
[12/16 04:01:03 d2.data.build]: Distribution of instances among all 13 categories:
|   category    | #instances   |   category    | #instances   |   category    | #instances   |
|:-------------:|:-------------|:-------------:|:-------------|:-------------:|:-------------|
| short_sleev.. | 1652         | long_sleeve.. | 948          | short_sleev.. | 6            |
| long_sleeve.. | 263          |     vest      | 296          |     sling     | 66           |
|    shorts     | 330          |   trousers    | 881          |     skirt     | 1048         |
| short_sleev.. | 464          | long_sleeve.. | 277          |  vest_dress   | 608          |
|  sling_dress  | 242          |               |              |               |              |
|     total     | 7081       

# Second Training & evaluate

In [ ]:
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("deepfashion_train_1_30000",)
cfg.DATASETS.TEST = ()
cfg.MODEL.WEIGHTS = '/content/output/model_final.pth'

cfg.SOLVER.IMS_PER_BATCH = 8
cfg.SOLVER.BASE_LR = 0.008 #0.001
cfg.SOLVER.WARMUP_ITERS = 500
cfg.SOLVER.MAX_ITER = 8000
cfg.SOLVER.STEPS = (500,2000,4000,5500,7000)
cfg.SOLVER.GAMMA = 0.5
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 13

cfg.TEST.EVAL_PERIOD = 2000

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[12/16 04:10:19 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

In [ ]:
# save weight
src = '/content/output/model_final.pth'
dst = '/content/drive/MyDrive/Colab Notebooks/model_final_202012161430.pth'
copyfile(src, dst)

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

evaluator = COCOEvaluator("deepfashion_val", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "deepfashion_val")
result=inference_on_dataset(trainer.model, val_loader, evaluator)

[12/16 06:28:44 d2.data.datasets.coco]: Loading /content/deepfashion2_validation.json takes 1.26 seconds.
[12/16 06:28:44 d2.data.datasets.coco]: Loaded 5000 images in COCO format from /content/deepfashion2_validation.json
[12/16 06:28:44 d2.data.common]: Serializing 5000 elements to byte tensors and concatenating them all ...
[12/16 06:28:45 d2.data.common]: Serialized dataset takes 65.80 MiB
[12/16 06:28:45 d2.evaluation.evaluator]: Start inference on 5000 images
[12/16 06:28:46 d2.evaluation.evaluator]: Inference done 11/5000. 0.0557 s / img. ETA=0:04:44
[12/16 06:28:51 d2.evaluation.evaluator]: Inference done 99/5000. 0.0555 s / img. ETA=0:04:40
[12/16 06:28:56 d2.evaluation.evaluator]: Inference done 188/5000. 0.0551 s / img. ETA=0:04:34
[12/16 06:29:01 d2.evaluation.evaluator]: Inference done 277/5000. 0.0550 s / img. ETA=0:04:28
[12/16 06:29:06 d2.evaluation.evaluator]: Inference done 368/5000. 0.0546 s / img. ETA=0:04:21
[12/16 06:29:11 d2.evaluation.evaluator]: Inference done 

# Third Training & evaluate

In [ ]:
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("deepfashion_train_1_30000",)
cfg.DATASETS.TEST = ()
cfg.MODEL.WEIGHTS = '/content/output/model_final.pth'

cfg.SOLVER.IMS_PER_BATCH = 8
cfg.SOLVER.BASE_LR = 0.0005 #0.001
cfg.SOLVER.WARMUP_ITERS = 500
cfg.SOLVER.MAX_ITER = 8000
cfg.SOLVER.STEPS = (1000,2000,4000,5500,7000)
cfg.SOLVER.GAMMA = 0.2
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 13

cfg.TEST.EVAL_PERIOD = 2000

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[12/16 06:37:39 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

In [ ]:
# save weight
src = '/content/output/model_final.pth'
dst = '/content/drive/MyDrive/Colab Notebooks/model_final_202012161700.pth'
copyfile(src, dst)

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

#predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("deepfashion_val", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "deepfashion_val")
result=inference_on_dataset(trainer.model, val_loader, evaluator)

[12/16 08:57:39 d2.data.datasets.coco]: Loading /content/deepfashion2_validation.json takes 1.56 seconds.
[12/16 08:57:40 d2.data.datasets.coco]: Loaded 5000 images in COCO format from /content/deepfashion2_validation.json
[12/16 08:57:40 d2.data.common]: Serializing 5000 elements to byte tensors and concatenating them all ...
[12/16 08:57:41 d2.data.common]: Serialized dataset takes 65.80 MiB
[12/16 08:57:41 d2.evaluation.evaluator]: Start inference on 5000 images
[12/16 08:57:42 d2.evaluation.evaluator]: Inference done 11/5000. 0.0544 s / img. ETA=0:04:37
[12/16 08:57:47 d2.evaluation.evaluator]: Inference done 99/5000. 0.0552 s / img. ETA=0:04:39
[12/16 08:57:52 d2.evaluation.evaluator]: Inference done 188/5000. 0.0551 s / img. ETA=0:04:33
[12/16 08:57:57 d2.evaluation.evaluator]: Inference done 277/5000. 0.0550 s / img. ETA=0:04:28
[12/16 08:58:02 d2.evaluation.evaluator]: Inference done 368/5000. 0.0547 s / img. ETA=0:04:21
[12/16 08:58:07 d2.evaluation.evaluator]: Inference done 

In [ ]:
result

OrderedDict([('bbox',
              {'AP': 40.28055884737942,
               'AP-long_sleeved_dress': 29.491517356796066,
               'AP-long_sleeved_outwear': 47.28243210142195,
               'AP-long_sleeved_shirt': 52.26691462698175,
               'AP-short_sleeved_dress': 44.56703418684515,
               'AP-short_sleeved_outwear': 0.0,
               'AP-short_sleeved_shirt': 67.84422282225464,
               'AP-shorts': 52.80149041293226,
               'AP-skirt': 55.22540390412154,
               'AP-sling': 3.1639441088258455,
               'AP-sling_dress': 24.287049114502814,
               'AP-trousers': 63.76639673141785,
               'AP-vest': 37.31552411014323,
               'AP-vest_dress': 45.635335539689386,
               'AP50': 51.20973634388544,
               'AP75': 46.21842125469101,
               'APl': 40.6244622284753,
               'APm': 35.70472604214353,
               'APs': 25.0})])

In [ ]:
import pickle
with open('model_val_result.pickle', 'wb') as handle:
    pickle.dump(result, handle)

In [ ]:
from shutil import copyfile

src = '/content/drive/MyDrive/Colab Notebooks/model_final_202012161700.pth'
dst = '/content/model_final.pth'
copyfile(src, dst)

'/content/model_final.pth'

# Forth Training & evaluate

In [ ]:
cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("deepfashion_train_1_100000",)
cfg.DATASETS.TEST = ()
cfg.MODEL.WEIGHTS = '/content/model_final.pth'

cfg.SOLVER.IMS_PER_BATCH = 8
cfg.SOLVER.BASE_LR = 0.001 #0.001
cfg.SOLVER.WARMUP_ITERS = 700
cfg.SOLVER.MAX_ITER = 12500
cfg.SOLVER.STEPS = (2000,4000,5500,7000,8500,10000,11500)
cfg.SOLVER.GAMMA = 0.2
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 256
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 13

cfg.TEST.EVAL_PERIOD = 2000

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
trainer.train()

[12/16 15:51:23 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

In [ ]:
# save weight
src = '/content/output/model_final.pth'
dst = '/content/drive/MyDrive/Colab Notebooks/model_final_202012170330.pth'
copyfile(src, dst)

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

evaluator = COCOEvaluator("deepfashion_val", cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "deepfashion_val")
result=inference_on_dataset(trainer.model, val_loader, evaluator)

[12/16 19:23:59 d2.data.datasets.coco]: Loading /content/deepfashion2_validation.json takes 1.44 seconds.
[12/16 19:23:59 d2.data.datasets.coco]: Loaded 5000 images in COCO format from /content/deepfashion2_validation.json
[12/16 19:24:00 d2.data.build]: Distribution of instances among all 13 categories:
|   category    | #instances   |   category    | #instances   |   category    | #instances   |
|:-------------:|:-------------|:-------------:|:-------------|:-------------:|:-------------|
| short_sleev.. | 1652         | long_sleeve.. | 948          | short_sleev.. | 6            |
| long_sleeve.. | 263          |     vest      | 296          |     sling     | 66           |
|    shorts     | 330          |   trousers    | 881          |     skirt     | 1048         |
| short_sleev.. | 464          | long_sleeve.. | 277          |  vest_dress   | 608          |
|  sling_dress  | 242          |               |              |               |              |
|     total     | 7081       